# Guaranteeing valid output syntax

Large language models are great at generating useful outputs, but they are not great at guaranteeing that those outputs follow a specific format. This can cause problems when we want to use the outputs of a language model as input to another system. For example, if we want to use a language model to generate a JSON object, we need to make sure that the output is valid JSON. This can be a real pain with standard APIs, but with `guidance` we can both accelerate inference speed and ensure that generated JSON is always valid.

This notebook shows how to generate a JSON object we know will have a valid format. The example used here is a generating a random character profile for a game, but the ideas are readily applicable to any scneeario where you want JSON output.

In [1]:
import guidance

# we use LLaMA here, but any GPT-style model will do
guidance.llm = guidance.llms.Transformers("huggyllama/llama-7b", device=0)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
program = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "description": "{{description}}",
    "name": "{{gen 'name'}}",
    "age": {{gen 'age' pattern='[0-9]+' stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra'}}",
    "strength": {{gen 'strength' pattern='[0-9]+' stop=','}},
    "items": [{{#geneach 'items' num_iterations=3}}
        "{{gen 'this'}}",{{/geneach}}
    ]
}```""")

# execute the prompt
out = program(description="A quick and nimble fighter.", valid_weapons=valid_weapons)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "description": " A quick and nimble fighter. ",
 "name": " Ranger ",
 "age": 20 ,
 "armor": " plate ",
 "weapon": " sword ",
 "class": " fighter ",
 "mantra": " I am the ranger. ",
 "strength": 10 ,
 "items": [ 
 " dagger ",
 " shield ",
 " bow ", 
 ]
}```

In [5]:
# we have now generated a valid json string that is mixure of generated and inserted text
print(str(out).split("```json")[1][:-3])


{
    "description": "A quick and nimble fighter.",
    "name": "Ranger",
    "age": 20,
    "armor": "plate",
    "weapon": "sword",
    "class": "fighter",
    "mantra": "I am the ranger.",
    "strength": 10,
    "items": [
        "dagger",
        "shield",
        "bow",
    ]
}


In [6]:
# and we also have a valid python dictionary
out.variables()

{'llm': <guidance.llms._transformers.Transformers at 0x7fae471f63e0>,
 'description': 'A quick and nimble fighter.',
 'valid_weapons': ['sword', 'axe', 'mace', 'spear', 'bow', 'crossbow'],
 'name': 'Ranger',
 'age': '20',
 'armor': 'plate',
 'weapon': 'sword',
 'class': 'fighter',
 'mantra': 'I am the ranger.',
 'strength': '10',
 'items': ['dagger', 'shield', 'bow']}

In [7]:
# ...that we could also serialize the dictionary to JSON
import json
json.dumps({k:v for k,v in out.variables().items() if k != "llm"})

'{"description": "A quick and nimble fighter.", "valid_weapons": ["sword", "axe", "mace", "spear", "bow", "crossbow"], "name": "Ranger", "age": "20", "armor": "plate", "weapon": "sword", "class": "fighter", "mantra": "I am the ranger.", "strength": "10", "items": ["dagger", "shield", "bow"]}'

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>